# 问题一求解代码

In [41]:
import numpy as np
import pandas as pd

In [42]:
# 数据读取
data = pd.read_excel('附件1近5年402家供应商的相关数据.xlsx', sheet_name='评价矩阵')
data = data[['供应商ID', '履约率', '稳定性系数', '需求供应比', '平均供货量', '最大供货能力', '供货总量', '供货频率', '超额供货率', '短缺供货率', '平均偏差值']] 

data = data[['履约率', '稳定性系数', '需求供应比', '平均供货量', '供货总量', '供货频率', '超额供货率', '短缺供货率','平均偏差值']]

data.head()

,履约率,稳定性系数,需求供应比,平均供货量,供货总量,供货频率,超额供货率,短缺供货率,平均偏差值
0,0.274725,0.280084,0.212121,1.960000,49,0.104167,0.109890,0.791209,0.874459
1,0.747368,0.228122,0.883495,3.845070,273,0.295833,0.315789,0.326316,0.498382
2,0.959799,0.717372,0.920092,68.785340,13138,0.795833,0.753769,0.145729,0.159045
3,0.320388,0.309929,0.089762,1.939394,64,0.137500,0.097087,0.796117,0.938289
4,0.938596,0.794324,1.057204,64.598131,6912,0.445833,0.798246,0.096491,0.073417


正向化处理

In [43]:
def normalize_indicators(data_matrix, indicator_types, ideal_values=None):
    """
    根据指标类型对数据进行正向化处理
    
    :param data_matrix: 二维数组，行为供应商，列为指标
    :param indicator_types: 列表，指示每个指标的类型
        'positive' - 极大型指标（越大越好）
        'negative' - 极小型指标（越小越好）
        'moderate' - 中间型指标（越接近某个值越好）
    :param ideal_values: 中间型指标的理想值列表
    :return: 正向化后的数据矩阵
    """
    if isinstance(data_matrix, pd.DataFrame):
        data_matrix = data_matrix.values
    normalized_matrix = np.zeros_like(data_matrix, dtype=float)
    n_indicators = data_matrix.shape[1]
    
    for j in range(n_indicators):
        col = data_matrix[:, j]
        indicator_type = indicator_types[j]
        
        if indicator_type == 'positive':
            # 极大型指标：无需处理
            normalized_matrix[:, j] = col
            
        elif indicator_type == 'negative':
            # 极小型指标：转化为极大型
            # 方法1：取倒数（适用于无零值的情况）
            # normalized_matrix[:, j] = 1 / (col + 1e-10)
            
            # 方法2：用最大值减去当前值（更常用）
            max_val = np.max(col)
            normalized_matrix[:, j] = max_val - col
            
        elif indicator_type == 'moderate':
            # 中间型指标：转化为极大型（越接近理想值越好）
            if ideal_values is None:
                raise ValueError("必须为中间型指标提供理想值")
                
            ideal = ideal_values[j]
            # 计算与理想值的绝对偏差
            deviation = np.abs(col - ideal)
            # 最大偏差（避免除零）
            max_dev = np.max(deviation) if np.max(deviation) > 0 else 1
            # 转化为极大型：偏差越小越好
            normalized_matrix[:, j] = 1 - deviation / max_dev
    
    return normalized_matrix

In [44]:
indicator_types = ['positive', 'positive', 'moderate', 'positive', 'positive', 'positive', 'negative' , 'negative', 'negative']
ideal_values = [None, None, 1, None, None, None, None, None, None]
normalize_matrix = normalize_indicators(data, indicator_types, ideal_values)

normalize_matrix


array([[0.27472527, 0.28008449, 0.21155523, ..., 0.79844322, 0.20879121,
        0.12504661],
       [0.74736842, 0.22812198, 0.88341145, ..., 0.59254386, 0.67368421,
        0.50112361],
       [0.95979899, 0.71737208, 0.92003504, ..., 0.15456449, 0.85427136,
        0.84046074],
       ...,
       [0.18055556, 0.14924161, 0.0650857 , ..., 0.78333333, 0.16666667,
        0.0205973 ],
       [0.43283582, 0.28817208, 0.33452229, ..., 0.71430348, 0.37313433,
        0.20450549],
       [0.10958904, 0.10823383, 0.03584234, ..., 0.88093607, 0.05479452,
        0.03186457]], shape=(402, 9))

topsis+熵权法

In [45]:
def topsis_method(data_matrix):
    """
    TOPSIS方法完整实现
    :param data_matrix: 二维数组，行为供应商，列为指标
    :return: 
        weights: 指标权重数组
        scores: 供应商TOPSIS得分数组
        rankings: 供应商排名数组
    """
    # 1. 矩阵归一化（向量归一化）
    # 计算每列的平方和
    col_sums = np.sqrt(np.sum(data_matrix**2, axis=0))
    # 避免除零错误
    col_sums[col_sums == 0] = 1e-10
    # 归一化矩阵
    norm_matrix = data_matrix / col_sums
    
    # 2. 利用熵权法确定指标权重
    # 计算指标比重
    p_matrix = norm_matrix / np.sum(norm_matrix, axis=0)
    
    # 计算信息熵
    m = data_matrix.shape[0]  # 供应商数量
    k = 1 / np.log(m)  # 熵计算系数
    e_j = np.zeros(data_matrix.shape[1])
    
    for j in range(data_matrix.shape[1]):
        col = p_matrix[:, j]
        # 避免log(0)错误
        with np.errstate(divide='ignore', invalid='ignore'):
            entropy = np.sum(col * np.log(col + 1e-10))
        e_j[j] = -k * entropy
    
    # 计算信息效用值
    d_j = 1 - e_j
    
    # 计算权重
    weights = d_j / np.sum(d_j)
    
    # 3. 构建加权规范化矩阵
    weighted_matrix = norm_matrix * weights
    
    # 4. 确定正负理想解
    positive_ideal = np.max(weighted_matrix, axis=0)
    negative_ideal = np.min(weighted_matrix, axis=0)
    
    # 5. 计算欧氏距离
    d_positive = np.sqrt(np.sum((weighted_matrix - positive_ideal)**2, axis=1))
    d_negative = np.sqrt(np.sum((weighted_matrix - negative_ideal)**2, axis=1))
    
    # 6. 计算相对接近度（TOPSIS得分）
    scores = d_negative / (d_positive + d_negative + 1e-10)
    
    # 7. 供应商排名
    rankings = np.argsort(-scores)  # 从高到低排序
    rankings = rankings + 1
    
    return weights, scores, rankings

In [46]:
weights, scores, rankings = topsis_method(normalize_matrix)

print(weights)
# print(sum(weights))
print(scores)
print(rankings)


[0.01811254 0.10549482 0.04721029 0.31431712 0.34434332 0.07365347
 0.01927591 0.02228658 0.05530595]
[0.0089439  0.02284792 0.04960605 0.00896183 0.04050905 0.00626667
 0.04863366 0.01172473 0.00730018 0.01091822 0.0236544  0.00632522
 0.02212815 0.01913588 0.00659929 0.02453673 0.01687666 0.0149985
 0.01053788 0.01308302 0.01227826 0.01167953 0.02318241 0.01552002
 0.03183115 0.01963767 0.02247944 0.01661847 0.00841208 0.02265807
 0.12957092 0.01820463 0.02441695 0.00636585 0.01975707 0.02295585
 0.12520451 0.01126562 0.01433212 0.08386599 0.01129029 0.01294682
 0.00727786 0.02172321 0.00855713 0.03437664 0.00805378 0.01670933
 0.01638013 0.02082626 0.00646634 0.01968172 0.02342547 0.02752017
 0.06441415 0.01390328 0.01092118 0.0075355  0.01262808 0.02304164
 0.00844709 0.02336032 0.00746145 0.02495295 0.02437325 0.02709129
 0.03922722 0.00697017 0.01778559 0.0139003  0.016372   0.00954166
 0.02237172 0.04208473 0.03038158 0.04187812 0.01360851 0.04008074
 0.00914299 0.06416032 0.022

In [47]:
print(f'前50个供应商排名：{rankings[:50]}')
print(f'前50个供应商得分：{scores[rankings[:50] - 1]}')


前50个供应商排名：[229 201 361 140 108 151 340 282 275 329 139 395 268 131 308 330 306 356
 194 348 352 307 143 247 126 266  31  37 294 346 374 284 365  40 364 338
 367  55  80 244 123 218  86 210   3   7 114 189 273 314]
前50个供应商得分：[0.64001948 0.62657912 0.60459286 0.58083736 0.46021392 0.3776139
 0.34552953 0.33459587 0.32678537 0.32213303 0.30804916 0.3011948
 0.28575021 0.28167146 0.27248658 0.27160576 0.26325336 0.26132111
 0.2359714  0.20121004 0.18718983 0.18149648 0.17101742 0.16819869
 0.16221204 0.13824148 0.12957092 0.12520451 0.11378229 0.11300688
 0.10805998 0.10542031 0.1051627  0.08386599 0.07592954 0.0746438
 0.07455312 0.06441415 0.06416032 0.06414722 0.06371093 0.0626515
 0.05262155 0.05023688 0.04960605 0.04863366 0.04699325 0.04558782
 0.04404309 0.04267809]


In [48]:
#? 美化输出
ranking_df = pd.DataFrame({
    '供应商排名': rankings[:],
    '得分': scores[rankings[:] - 1]
})

# 设置索引从1开始
ranking_df.index = range(1, 402 + 1)

# 设置列名
ranking_df.columns = ['供应商编号', 'TOPSIS得分']

# 设置显示格式
pd.set_option('display.float_format', lambda x: '{:.6f}'.format(x))

# 打印美化后的结果
print("前50名供应商排名及得分情况：")
print("=" * 50)
print(ranking_df)
print("=" * 50)

前50名供应商排名及得分情况：
     供应商编号  TOPSIS得分
1      229  0.640019
2      201  0.626579
3      361  0.604593
4      140  0.580837
5      108  0.460214
..     ...       ...
398    402  0.005930
399    400  0.005911
400    372  0.005855
401    251  0.005669
402    355  0.005645

[402 rows x 2 columns]


In [49]:
try:   
    !jupyter nbconvert --to python q1.ipynb
    # python即转化为.py，script即转化为.html
except:
    pass

[NbConvertApp] Converting notebook q1.ipynb to python
[NbConvertApp] Writing 4717 bytes to q1.py


In [50]:
# 导出为图表
data.head()


,履约率,稳定性系数,需求供应比,平均供货量,供货总量,供货频率,超额供货率,短缺供货率,平均偏差值
0,0.274725,0.280084,0.212121,1.960000,49,0.104167,0.109890,0.791209,0.874459
1,0.747368,0.228122,0.883495,3.845070,273,0.295833,0.315789,0.326316,0.498382
2,0.959799,0.717372,0.920092,68.785340,13138,0.795833,0.753769,0.145729,0.159045
3,0.320388,0.309929,0.089762,1.939394,64,0.137500,0.097087,0.796117,0.938289
4,0.938596,0.794324,1.057204,64.598131,6912,0.445833,0.798246,0.096491,0.073417
